Imports and functions

In [77]:
import os
import netCDF4 as nc4
import numpy as np

User inputs

In [78]:
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/06/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/07/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/10/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/11/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/12/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/02/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/04/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/06/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2015/07/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2016/01/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2016/04/test'
indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/03'
THRESHOLD = 40.0  #dBZ
SLOPE_LO = 0.5
SLOPE_HI = 2.0
VERT_RES = 0.125  #km

In [79]:
#for fname in os.listdir(indir):
#   print fname

In [80]:
#fname = 'GPM2Ku4_uw1_20140630.202105_to_20140630.202518_001916_NAM.nc'
#fname = 'GPM-2Ku.007185.20150604.111137.nc'
fname = 'GPM-2Ku.000457.20140329.012549.nc'

In [81]:
    #ncid = nc4.Dataset(indir+'/'+fname,'a')
    ncid = nc4.Dataset(indir+'/'+fname,'r')

In [82]:
    refl = np.array(np.squeeze(ncid.variables['refl']))
    (nalt,nlat,nlon) = refl.shape
    refl_missing = ncid.variables['refl'].missing_value
    #refl[(refl==refl_missing)] = np.nan

In [83]:
print nalt, nlat, nlon
print refl.shape

176 4 4
(176, 4, 4)


In [84]:
    rt = np.array(ncid.variables['rain_type'])
    rt_missing = ncid.variables['rain_type'].missing_value
    #rt[(rt==rt_missing)] = np.nan

In [85]:
rt.shape

(1, 4, 4)

In [86]:
    rt_uw = np.array(rt, copy=True)

In [87]:
    #rt_uw = np.ma.copy(rt)

    change rain_type to stratiform in rt_uw if:
    1. rain_type = convective and
    2. max_refl in column >= THRESHOLD and
    3. (max_refl - dbz@(max_refl_ht-1km)/deltaHt > abs(SLOPE_LO)
    4. (max_refl - dbz@(max_refl_ht+1km)/deltaHt > abs(SLOPE_HI)

In [88]:
    # find max_refl and max_refl_ht at each lat/lon
    max_refl = np.zeros(rt.shape,dtype=float)
    max_refl = np.amax(refl,axis=1)
    max_refl_bin = np.zeros(rt.shape,dtype=float)
    max_refl_bin = np.argmax(refl,axis=1)
    max_refl_ht = max_refl_bin*VERT_RES

In [89]:
    # find bin of highest good refl at each lat/lon
    maxht = np.zeros((nlat,nlon))+refl_missing
    for lev in range(nalt-1,-1,-1):
        maxht = np.where(np.logical_and( (refl[lev,:,:] != refl_missing),
                                         (maxht == refl_missing) ), 
                         lev*VERT_RES, maxht ) 

In [90]:
maxht

array([[  2.   ,   2.   ,   2.   ,   1.75 ],
       [-99.   ,   2.   , -99.   ,   1.75 ],
       [  1.875,   1.875, -99.   , -99.   ],
       [  1.875, -99.   , -99.   , -99.   ]])

In [76]:
np.flipud(maxht)

array([[  1.875, -99.   , -99.   , -99.   ],
       [  1.875,   1.875, -99.   , -99.   ],
       [-99.   ,   2.   , -99.   ,   1.75 ],
       [  2.   ,   2.   ,   2.   ,   1.75 ]])

In [67]:
lev*VERT_RES+VERT_RES

2.125

In [ ]:
a = refl[:,lev,:,:] != refl_missing

In [ ]:
a

In [ ]:
a.shape

In [ ]:
b = maxht == refl_missing

In [ ]:
b.shape

In [ ]:
        maxht = np.where(np.logical_and( (refl[:,lev,:,:] != refl_missing),
                                         (maxht == refl_missing) ) )

In [ ]:
maxht.shape

In [ ]:
    # find slopes
    if max_refl_bin >= 8:
        refl_minus1 = refl[max_refl_bin-8]
        deltaHt_minus1 = 1
    else:
        refl_minus1 = refl[0]
        deltaHt_minus1 = max_refl_ht
        
    if refl[max_refl_bin+8] != refl_missing:
        refl_plus1 = refl[max_refl_bin+8]
        deltaHt_plus1 = 1
    else:
        

In [ ]:
    change = np.logical_and(np.logical_and(rt==2,max_refl>=THRESHOLD),
                            np.logical_and(max_refl_ht>=MIN_HT,
                                           max_refl_ht<=MAX_HT))

In [ ]:
    rt_uw[change==True]=1

    Add new raintype field to input netcdf file

In [ ]:
    rt_uw_id = ncid.createVariable('rain_type_uw','f4',
                                  ('time','latitude','longitude'),zlib=True)
    #rt_uw_id = ncid.createVariable('rain_type_uw','f4',
    #                              ('time','latitude','longitude'))
    rt_uw_id.units = "none"
    rt_uw_id.long_name = "Rain Type UW"
    rt_uw_id.stratiform = 1
    rt_uw_id.convective = 2
    rt_uw_id.other = 3
    rt_uw_id.missing_value = rt_missing
    rt_uw_id[:,:,:] = rt_uw
    ncid.close()

In [ ]:
ncid.close()